In [ ]:
%%bash
[[ -z "$COLAB_GPU" ]] && exit 0
apt install subversion >> log
pip install pyarts typhon >> log
svn co -q https://arts.mi.uni-hamburg.de/svn/rt/arts-xml-data/trunk/spectroscopy/Perrin/ >> log

In [36]:
"""Calculate and plot absorption cross sections."""
import glob
from os.path import basename

from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from pyarts.workspace import Workspace
from typhon.plots import styles
from typhon.utils import Timer

plt.style.use(styles("typhon"))

PATH_TO_PERRIN = "./Perrin/"


class ArtsCalc:
    def __init__(self, verbosity=0):
        self.species = "H2O"
        self.workspace = None
        self.ymax = None
        self.verbosity = verbosity
        self.nfrequencies = 5000
        self.f_grid = None
        self.fmin = 10e9
        self.fmax = 2000e9
        self.abs_xsec = None
        self.temperature = 300
        self.pressure = 800 * 100

    def plot_absorption(self, temperature, pressure, species, nfrequencies):
        # Update parameters
        verbosity = 0
        self.temperature = temperature
        self.pressure = pressure * 100

        # Call ARTS to calculate absorption cross sections
        if nfrequencies != self.nfrequencies or species != self.species:
            self.species = species
            self.nfrequencies = nfrequencies
            self.setup_absxsec()
            self.ymax = None

        self.calculate_absxsec()

        if self.ymax is None:
            self.ymax = np.max(self.abs_xsec)

        # Plot the results.

        fig, ax = plt.subplots()
        plt.subplots_adjust(bottom=0.25)

        ax.plot(self.f_grid / 1e9, self.abs_xsec, rasterized=True)

        ax.set_xlim(self.f_grid.min() / 1e9, self.f_grid.max() / 1e9)
        ax.set_ylim(0, self.ymax)
        ax.set_xlabel("Frequency [GHz]")
        ax.set_ylabel(r"Abs. cross section [$\sf m^2$]")
        ax.set_title(
            f"{self.species} - P: {self.pressure/100} hPa - T: {self.temperature:0.0f} K"
        )

        plt.show()  # Open an interactive figure

    def setup_absxsec(
        self,
        lineshape="LP",
        normalization="RQ",
        verbosity=0,
    ):
        """Setup absorption cross sections.

        Parameters:
            lineshape (str): Line shape model.
            normalization (str): Line shape normalization factor.
            verbosity (int): Set ARTS verbosity (``0`` prevents all output).

        Returns:
            Workspace: ARTS Workspace.
        """
        # Create ARTS workspace and load default settings
        self.workspace = Workspace(verbosity=verbosity)
        ws = self.workspace
        ws.execute_controlfile("general/general.arts")
        ws.execute_controlfile("general/continua.arts")
        ws.execute_controlfile("general/agendas.arts")
        ws.verbositySetScreen(ws.verbosity, verbosity)

        # We do not want to calculate the Jacobian Matrix
        ws.jacobianOff()

        # Agenda for scalar gas absorption calculation
        ws.Copy(ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA)

        # Define absorption species
        ws.abs_speciesSet(species=[self.species])
        ws.ArrayOfIndexSet(ws.abs_species_active, [0])

        # Line catalogue: Perrin or HITRAN
        ws.ReadSplitARTSCAT(
            basename=PATH_TO_PERRIN,
            fmin=0.9 * self.fmin,
            fmax=1.1 * self.fmax,
            globalquantumnumbers="",
            localquantumnumbers="",
            ignore_missing=0,
        )
        ws.abs_lines_per_speciesCreateFromLines()

        # Set the lineshape function for all calculated tags
        ws.abs_linesSetLineShapeType(ws.abs_lines, lineshape)
        ws.abs_linesSetCutoff(ws.abs_lines, "None", 0.0)
        ws.abs_linesSetNormalization(ws.abs_lines, normalization)

        # isotop
        ws.isotopologue_ratiosInitFromBuiltin()

        # Atmospheric settings
        ws.AtmosphereSet1D()

        # Create a frequency grid
        ws.VectorNLinSpace(ws.f_grid, self.nfrequencies, self.fmin, self.fmax)

        return ws

    def calculate_absxsec(self):
        """Calculate absorption cross sections.

        Returns:
            ndarray, ndarray: Frequency grid [Hz], Abs. cross sections [m^2]
        """
        # Setting the pressure, temperature and vmr
        ws = self.workspace
        ws.NumericSet(ws.rtp_pressure, float(self.pressure))  # [Pa]
        ws.NumericSet(ws.rtp_temperature, float(self.temperature))  # [K]
        ws.VectorSet(ws.rtp_vmr, np.array([1.0]))  # [VMR]
        ws.Touch(ws.abs_nlte)

        ws.AbsInputFromRteScalars()

        # Calculate absorption cross sections
        ws.lbl_checkedCalc()
        ws.abs_xsec_agenda_checkedCalc()

        ws.abs_xsec_per_speciesInit()
        ws.abs_xsec_per_speciesAddLines()

        self.f_grid = ws.f_grid.value
        self.abs_xsec = ws.abs_xsec_per_species.value[0]

In [41]:
ac = ArtsCalc()
interactive_plot = interactive(
    ac.plot_absorption,
    species=sorted(
        [basename(s).split(".")[0] for s in glob.iglob("./Perrin/*.xml.gz")]
    ),
    nfrequencies=(1000, 10000, 1),
    temperature=(200, 400, 1.0),
    pressure=(0, 1000, 1.0),
)

for i in interactive_plot.children:
    i.continuous_update = False

output = interactive_plot.children[-1]
interactive_plot

interactive(children=(FloatSlider(value=300.0, continuous_update=False, description='temperature', max=400.0, …